Начинаем работу с импортов всего, что нам пригодится:

In [1369]:
import pandas as pd
import numpy as np
import nltk
import string
import spacy

## 1. dataset

работаем мы сегодня с датасетом Digital Innovation Research Dataset (1983-2019) (https://data.mendeley.com/datasets/gpr2phd9mk/1) - списком публикаций по теме цифровых инноваций.

из него нам понадобятся колонки с кратким содержанием и тегами, заданными авторами; извлечем все это в датафрейм и заодно заполним НаНы пустотами

In [1370]:
data_1 = pd.read_csv('digital innovation dataset.csv', usecols = ['Abstract','Author Keywords'])
data_1 = data_1.fillna('')
data_1.head()

,Abstract,Author Keywords
0,Digital India is the dream project for the gov...,Chi-Square; Correlation; Cross-tab; Digitaliza...
1,"Purpose: As various scholars have pointed out,...",Antecedents; Digital; Digital innovation manag...
2,Purpose: The purpose of this paper is to revie...,Digital innovation; Information technology; In...
3,Purpose: Many start-ups are in search of coope...,Cooperation behaviour; Corporate start-up; Dig...
4,Purpose: Digital transformations are changing ...,Business processes; Conceptual modelling; Digi...


красивое. Выкинем отсюда все, что не имеет краткого содержания и/или авторских тегов.

In [1371]:
data_clean = data_1[~(data_1['Abstract'] == '[No abstract available]') & ~(data_1['Author Keywords'] == '')]
data_clean.head()

,Abstract,Author Keywords
0,Digital India is the dream project for the gov...,Chi-Square; Correlation; Cross-tab; Digitaliza...
1,"Purpose: As various scholars have pointed out,...",Antecedents; Digital; Digital innovation manag...
2,Purpose: The purpose of this paper is to revie...,Digital innovation; Information technology; In...
3,Purpose: Many start-ups are in search of coope...,Cooperation behaviour; Corporate start-up; Dig...
4,Purpose: Digital transformations are changing ...,Business processes; Conceptual modelling; Digi...


немного поперебираем, чтобы посмотреть, сколько единиц нам нужно взять, чтобы получить где-то 3-5 тысяч токенов - токенизировать будем с помощью нлтк.

In [1372]:
counta = 0
for i in data_clean.Abstract[:20]:
    counta += len(nltk.word_tokenize(i))
counta

4546

ага, 20 штук подойдет, возьмем с запасом, чтобы потом можно было что-нибудь неподходящее удалить и токенов все равно было бы достаточно. Вытащим эти 20 штук в новый датафрейм, заодно обновим индексы, чтобы поправить пропавшие при очистке.

In [1373]:
data_piece = data_clean.copy()[:20].reset_index(drop=True)
data_piece.head()

,Abstract,Author Keywords
0,Digital India is the dream project for the gov...,Chi-Square; Correlation; Cross-tab; Digitaliza...
1,"Purpose: As various scholars have pointed out,...",Antecedents; Digital; Digital innovation manag...
2,Purpose: The purpose of this paper is to revie...,Digital innovation; Information technology; In...
3,Purpose: Many start-ups are in search of coope...,Cooperation behaviour; Corporate start-up; Dig...
4,Purpose: Digital transformations are changing ...,Business processes; Conceptual modelling; Digi...


какой-то датафрейм у нас есть - но, очевидно, его никуда толком не скормить. Препроцессим!

### preprocessing

Начнем с определения парочки нужных функций.

эта функция позволит нам разбить авторские теги из перечисления в список, заодно опустив регистр.

In [1374]:
def author_keywords_listify(some_string):
    #input: string
    #output: list of strings
    stringlist = some_string.lower().split('; ')
    return stringlist

эта функция позволит нам привести к приличному виду содержания: выкинуть оформительные "цели работы, методология, задачи...", чтобы они не всплыли потом в качестве ключевых слов, отбросить концевой копирайт и опустить регистр.

In [1375]:
def delete_misc(some_string):
    #input: string
    #output: string
    string_0 = some_string.replace('Purpose: ','').split(' ©')[0]
    string_1 = string_0.replace('Design/methodology/approach: ','').replace('Findings: ','')
    string_2 = string_1.replace('Originality/value: ','')
    return string_2.lower()

пройдемся обеими функциями по нашим данным.

In [1376]:
clean_abstract = data_piece['Abstract'].map(delete_misc)
data_piece['Abstract'] = clean_abstract

In [1377]:
keywords_listified = data_piece['Author Keywords'].map(author_keywords_listify)
data_piece['Author Keywords'] = keywords_listified

In [1378]:
data_piece.head()

,Abstract,Author Keywords
0,digital india is the dream project for the gov...,"[chi-square, correlation, cross-tab, digitaliz..."
1,"as various scholars have pointed out, the expo...","[antecedents, digital, digital innovation mana..."
2,the purpose of this paper is to review and int...,"[digital innovation, information technology, i..."
3,many start-ups are in search of cooperation pa...,"[cooperation behaviour, corporate start-up, di..."
4,"digital transformations are changing society, ...","[business processes, conceptual modelling, dig..."


отлично! с этой проблемой разобрались, и теперь имеем нормальные содержания и списки авторских тегов.

тем не менее, авторские теги это, конечно, хорошо - но никто не гарантирует, что они являются ключевыми словами, т.е. будут в содержании. Это надо проверить.

In [1379]:
def check_abstract(keyword_list, abstract):
    #input: list of keywords, string
    #output: list of keywords
    removal_list = []
    for keyword in keyword_list:
        if keyword in abstract:
            pass
        else:
            removal_list.append(keyword)
    if bool(removal_list)==True:
        new_keywords = [x for x in keyword_list if x not in removal_list]
        removal_list = []
        return new_keywords
    else:
        return keyword_list

пройдемся этим по каждому ряду датафрейма, оставляя только те авторские теги, которые встречались в тексте; их добавим в список.

In [1380]:
checking_list = []
for row in range(len(data_piece)):
    checking_list.append(check_abstract(data_piece['Author Keywords'][row], data_piece['Abstract'][row]))

из первого ряда вообще никакие авторские теги не являются ключевыми словами - список пустой. Удаляем его, а оставшиеся 19 штук пересохраняем в финальный красивый датафрейм. С ним мы и будем дальше работать - так что снова обновим индексы, а то мало ли что мы там удаляли.

In [1381]:
data = data_piece.drop(0).reset_index(drop=True)
data['Author Keywords'] = checking_list[1:]

In [1382]:
data.head()

,Abstract,Author Keywords
0,"as various scholars have pointed out, the expo...","[digital, innovation, outcomes, processes]"
1,the purpose of this paper is to review and int...,"[information technology, innovation, literatur..."
2,many start-ups are in search of cooperation pa...,"[start-up cooperation, start-up performance]"
3,"digital transformations are changing society, ...","[business processes, innovation]"
4,the article identifies how collaborations with...,"[digital innovation, industry 4.0, innovation ..."


почти готово! осталось только лемматизировать - желательно и то, и другое, потому что искать нелемматизованные теги в лемматизованном тексте было бы не особо эффективно.

так как мы работаем с английскими текстами (слава богу), для лемматизации можем использовать spacy с токенизацией через nltk - возьмем лучшее из обоих миров! И очистим от пунктуации заодно.

работать будем в две функции - первая, маленькая вспомогательная, делает основную работу по лемматизации текста; вторая, оберточная, определяет, дали ей содержание или список тегов, и в зависимости от этого формирует список-выдачу

In [1383]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lem_helper(text):
    tokenized = nltk.word_tokenize(text)
    tokens = list(filter(lambda token: token not in string.punctuation, tokenized))
    doc = nlp(' '.join(tokens))
    return ' '.join([token.lemma_ for token in doc])
    
def lemmatization(obj, flag):
    all_lems = []
    if flag == 'abstract':
        all_lems = lem_helper(obj)       
    elif flag == 'keywords':
        for el in obj:
            all_lems.append(lem_helper(el))
    return all_lems

In [1384]:
lemmatization(data['Author Keywords'][7], flag='keywords')

['data - drive motion correction', 'digital innovation', 'pet', 'real - time']

ах, как печально. Спейси засчитывает дефис за разделитель слов! Это сильно портит нам настроение. Что будем делать?

ну, например, проконсультируемся с его документацией (https://spacy.io/usage/linguistic-features) - а если она даст нам даже кусок кода, которым можно исправить методы определения границ слов, будет вообще прекрасно. Даст же?

In [1385]:
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer

In [1386]:
lemmatization(data['Author Keywords'][7], flag='keywords')

['data-driven motion correction', 'digital innovation', 'pet', 'real-time']

боже, храни эту библиотеку и авторов ее документации.

пойдемте все наконец лемматизируем и начнем работать.

In [1387]:
data['abstract_lemmatized'] = data['Abstract'].map(lambda p: lemmatization(p, flag='abstract'))
data['Author_keys_lemmatized'] = data['Author Keywords'].map(lambda p: lemmatization(p, flag='keywords'))

In [1388]:
data.head()

,Abstract,Author Keywords,abstract_lemmatized,Author_keys_lemmatized
0,"as various scholars have pointed out, the expo...","[digital, innovation, outcomes, processes]",as various scholar have point out the exponent...,"[digital, innovation, outcome, process]"
1,the purpose of this paper is to review and int...,"[information technology, innovation, literatur...",the purpose of this paper be to review and int...,"[information technology, innovation, literatur..."
2,many start-ups are in search of cooperation pa...,"[start-up cooperation, start-up performance]",many start-up be in search of cooperation part...,"[start-up cooperation, start-up performance]"
3,"digital transformations are changing society, ...","[business processes, innovation]",digital transformation be change society and t...,"[business process, innovation]"
4,the article identifies how collaborations with...,"[digital innovation, industry 4.0, innovation ...",the article identify how collaboration with st...,"[digital innovation, industry 4.0, innovation ..."


препроц закончен!

## 2. own keywords and gold standard

Начинаем медленную и печальную работу по ручной разметке. Я закомментировала эту ячейку, но поверьте - она выдала мне содержания всех наших образцов, и я подобрала ко всем теги. Ух.

In [1389]:
#for each in data['Abstract']:
    #print(each)
    #print('\n')

In [1390]:
own_keys = []
own_keys.append(['innovation', 'business process', 'innovation input', 'innovation process', 'innovation outcome', 'digital innovation'])
own_keys.append(['information technology', 'literature review', 'innovation'])
own_keys.append(['cooperation', 'start-up cooperation', 'cooperation behavior', 'incumbent firms', 'cooperation model'])
own_keys.append(['digital change', 'innovation', 'cooperative model'])
own_keys.append(['digital innovation', 'r&d collaboration' , 'open innovation', 'industry 4.0', 'innovation practice', 'startup'])
own_keys.append(['technology development in agriculture', 'agriculture', 'smart farm', 'digital innovation', 'digiware', 'advisory service', 'farm management'])
own_keys.append(['public welfare', 'digital innovation', 'environmental public welfare project'])
own_keys.append(['data-driven innovation', 'hardware-driven motion correction','data-driven motion correction', 'ddmc', 'real-time ddmc', 'pet', 'motion correction'])
own_keys.append(['communication technology', 'computer literacy', 'digital innovation in teaching', 'educational process', 'digitally mature school project'])
own_keys.append(['mental disorder', 'mental health', 'mental health care', 'low resource setting', 'digital intervention'])
own_keys.append(['ethical issue', 'ethical research', 'mhealth', 'autonomy'])
own_keys.append(['healthcare provision', 'healthcare', 'health data', 'europe', 'cooperation', 'digital innovation'])
own_keys.append(['automation', 'data science', 'journalistic field', 'technological field', 'digital innovation', 'field theory'])
own_keys.append(['technology design', 'empowerment', 'participatory practice', 'social innovation'])
own_keys.append(['business model', 'incumbent', 'digital innovation', 'digital technology', 'dts', 'incumbent firm'])
own_keys.append(['crowdsource', 'digital innovation ecosystem', 'management field'])
own_keys.append(['information technology', 'it', 'patent invention', 'innovation process', 'knowledge recombinant intensity', 'knowledge recombinant diversity'])
own_keys.append(['digital innovation', 'knowledge integration', 'cross-domain collaboration', 'knowledge domain', 'coordination'])
own_keys.append(['operating model', 'digital innovation', 'fuel loyalty program', 'loyalty program platform', 'slpp'])
assert len(own_keys) == len(data)
data['own_keywords'] = list(map(lambda p: lemmatization(p, flag='keywords'), own_keys))

убедившись, что мы не пропустили ничего и длина списка тегов равна длине датафрейма, составляем золотой стандарт - совокупность наших и авторских тегов.

In [1391]:
g_standard = []
for num in range(len(data)):
    g_standard.append(set(data['Author_keys_lemmatized'][num]).union(set(data['own_keywords'][num])))
data['gold_standard'] = [list(x) for x in g_standard]

In [1392]:
data.head()

,Abstract,Author Keywords,abstract_lemmatized,Author_keys_lemmatized,own_keywords,gold_standard
0,"as various scholars have pointed out, the expo...","[digital, innovation, outcomes, processes]",as various scholar have point out the exponent...,"[digital, innovation, outcome, process]","[innovation, business process, innovation inpu...","[innovation, outcome, digital, innovation inpu..."
1,the purpose of this paper is to review and int...,"[information technology, innovation, literatur...",the purpose of this paper be to review and int...,"[information technology, innovation, literatur...","[information technology, literature review, in...","[innovation, information technology, literatur..."
2,many start-ups are in search of cooperation pa...,"[start-up cooperation, start-up performance]",many start-up be in search of cooperation part...,"[start-up cooperation, start-up performance]","[cooperation, start-up cooperation, cooperatio...","[cooperation, start-up performance, start-up c..."
3,"digital transformations are changing society, ...","[business processes, innovation]",digital transformation be change society and t...,"[business process, innovation]","[digital change, innovation, cooperative model]","[business process, innovation, digital change,..."
4,the article identifies how collaborations with...,"[digital innovation, industry 4.0, innovation ...",the article identify how collaboration with st...,"[digital innovation, industry 4.0, innovation ...","[digital innovation, r d collaboration, open i...","[innovation practice, innovation ecosystem, di..."


бинго.

## 3. autokeywords

### rake

наш первый кандидат - РАКЕ, который я буду писать так, потому что мне лень переключать раскладку. Скормим ему лемматизированные содержания без стоп-слов.

In [1393]:
import RAKE
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [1394]:
rake = RAKE.Rake(stop)

In [1395]:
rake_kw = []
for num in range(len(data)):
    rake_kw.append([x[0] for x in rake.run(data['abstract_lemmatized'].values[num], minCharacters=2, maxWords=3, minFrequency=1.5)])
data['RAKE'] = rake_kw

### textrank

следующий кандидат - textrank. Механизм работы с ним примерно тот же - передать стоп-слова и лемматизированный текст.

In [1396]:
from summa import keywords

In [1397]:
textrank = []
for num in range(len(data)):
    textrank.append([x[0] for x in keywords.keywords(data['abstract_lemmatized'].values[num], language='english', additional_stopwords=stop, scores=True)])
data['textrank'] = textrank

### yake

а кто же это такой? Да это же УАКЕ!

https://github.com/LIAAD/yake

In [1398]:
import yake
yake_kw_extractor = yake.KeywordExtractor(lan='en', dedupLim=0.95, features=None)

In [1399]:
yake_kw = []
for num in range(len(data)):
    keywords = yake_kw_extractor.extract_keywords(data['abstract_lemmatized'].values[num])
    yake_kw.append([x[0].lower() for x in keywords])
data['YAKE'] = yake_kw

## 4

Как мы будем делать частеречные шаблоны? Как и всегда - с помощью какой-нибудь маленькой, но трудолюбивой функции, которая нам все сделает. Правда, написать ее придется нам самим.

Механизм простой, как банка - токенизируем переданные слова, определяем их часть речи, склеиваем в сочетания частей речи, определяемых универсальной системой тегов, чтобы красивее было; сет нам нужен, чтобы не повторяться.

In [1400]:
from nltk import pos_tag, word_tokenize

def pos_helper(wordlist):
    pos_set = set()
    pos_tokens = [word_tokenize(x) for x in wordlist]
    for y in pos_tokens:
        pos_set.add('+'.join([x[1] for x in nltk.pos_tag(y, tagset='universal')]))
    return pos_set

In [1401]:
data['mold'] = data['gold_standard'].map(pos_helper)

In [1402]:
data.head()

,Abstract,Author Keywords,abstract_lemmatized,Author_keys_lemmatized,own_keywords,gold_standard,RAKE,textrank,YAKE,mold
0,"as various scholars have pointed out, the expo...","[digital, innovation, outcomes, processes]",as various scholar have point out the exponent...,"[digital, innovation, outcome, process]","[innovation, business process, innovation inpu...","[innovation, outcome, digital, innovation inpu...","[digital technology, purpose]","[exponential growth, compress anticipate, inno...","[special issue paper, special issue accord, sp...","{NOUN, NOUN+NOUN, ADJ+NOUN}"
1,the purpose of this paper is to review and int...,"[information technology, innovation, literatur...",the purpose of this paper be to review and int...,"[information technology, innovation, literatur...","[information technology, literature review, in...","[innovation, information technology, literatur...","[group level, innovation, role, individual, to...","[publish, integrate, integrative, integration,...","[research publish focus, support innovation pu...","{NOUN, NOUN+NOUN}"
2,many start-ups are in search of cooperation pa...,"[start-up cooperation, start-up performance]",many start-up be in search of cooperation part...,"[start-up cooperation, start-up performance]","[cooperation, start-up cooperation, cooperatio...","[cooperation, start-up performance, start-up c...","[cooperation behavior, start-, use, build, per...","[study, cooperation, base, start, factor, scal...","[past study identify, lack empirical research,...","{NOUN, NOUN+NOUN, ADJ+NOUN}"
3,"digital transformations are changing society, ...","[business processes, innovation]",digital transformation be change society and t...,"[business process, innovation]","[digital change, innovation, cooperative model]","[business process, innovation, digital change,...","[certain level, case company, change, digitali...","[change society, digital transformation, accor...","[process maturity level, process maturity mode...","{NOUN, NOUN+NOUN, ADJ+NOUN}"
4,the article identifies how collaborations with...,"[digital innovation, industry 4.0, innovation ...",the article identify how collaboration with st...,"[digital innovation, industry 4.0, innovation ...","[digital innovation, r d collaboration, open i...","[innovation practice, innovation ecosystem, di...","[startup, collaboration, industry 4, brazil]","[article identify, low maturity stage, theoret...","[international innovation center, open innovat...","{NOUN, NOUN+NOUN, ADJ+NOUN, NOUN+NOUN+NOUN, NO..."


отлично, есть молды (шаблоны) частеречных сочетаний - для каждого текста определенные относительно золотого стандарта для этого текста. Но что будет, если определять их относительно _всех золотых стандартов_, которые мы встретили в выборке? Это ведь все научные статьи, еще и по одной и той же теме. Но все авторские теги создавались разными авторами...

посмотрим. Для того, чтобы посмотреть, сделаем общий сет всех частеречных сочетаний, которые встретились в нашей выборке.

In [1403]:
full_mold_set = set()
for num in range(len(data)):
    full_mold_set |= data['mold'][num]
full_mold_set

{'ADJ',
 'ADJ+ADJ+NOUN+NOUN',
 'ADJ+NOUN',
 'ADJ+NOUN+ADP+NOUN',
 'ADJ+NOUN+NOUN',
 'ADV+ADJ+NOUN+NOUN',
 'NOUN',
 'NOUN+NOUN',
 'NOUN+NOUN+ADP+NOUN',
 'NOUN+NOUN+NOUN',
 'NOUN+NUM',
 'NOUN+VERB',
 'PRON'}

вот и ладушки. пойдемте теперь с помощью этих молдов фильтровать выдачи моделек.

как мы будем это делать? все так же, все так же...

In [1404]:
def compare_helper(dataframe, model):
    molded_loc = []
    molded_full = []
    for num in range(len(dataframe)):
        molded_loc_num = []
        molded_full_num = []
        for keywords in dataframe[model][num]:
            pos_tokens = word_tokenize(keywords)
            tag_group = '+'.join([x[1] for x in nltk.pos_tag(pos_tokens, tagset='universal')])
            if tag_group in dataframe['mold'][num]:
                molded_loc_num.append(keywords)
                molded_full_num.append(keywords)
            elif tag_group in full_mold_set:
                molded_full_num.append(keywords)         
        molded_loc.append(molded_loc_num)
        molded_full.append(molded_full_num)
    return molded_loc, molded_full

локальный молд = частеречные сочетания, встреченные в золотом стандарте конкретного текста, с которым мы сейчас работаем; полный молд = все частеречные сочетания, встреченные в золотых стандартах выборки. Обрабатываем все выдачи моделек и пропускаем дальше только тех кандидатов, которые подходят под наши молды.

In [1405]:
data = data.assign(RAKE_molded_loc = compare_helper(data, 'RAKE')[0],
            RAKE_molded_full = compare_helper(data, 'RAKE')[1],
            YAKE_molded_loc = compare_helper(data, 'YAKE')[0],
            YAKE_molded_full = compare_helper(data, 'YAKE')[1],
            textrank_molded_loc = compare_helper(data, 'textrank')[0],
            textrank_molded_full = compare_helper(data, 'textrank')[1])

In [1406]:
data.head()

,Abstract,Author Keywords,abstract_lemmatized,Author_keys_lemmatized,own_keywords,gold_standard,RAKE,textrank,YAKE,mold,RAKE_molded_loc,RAKE_molded_full,YAKE_molded_loc,YAKE_molded_full,textrank_molded_loc,textrank_molded_full
0,"as various scholars have pointed out, the expo...","[digital, innovation, outcomes, processes]",as various scholar have point out the exponent...,"[digital, innovation, outcome, process]","[innovation, business process, innovation inpu...","[innovation, outcome, digital, innovation inpu...","[digital technology, purpose]","[exponential growth, compress anticipate, inno...","[special issue paper, special issue accord, sp...","{NOUN, NOUN+NOUN, ADJ+NOUN}","[digital technology, purpose]","[digital technology, purpose]","[special issue, digital technology, innovation...","[special issue paper, special issue accord, sp...","[exponential growth, compress anticipate, inno...","[exponential growth, compress anticipate, inno..."
1,the purpose of this paper is to review and int...,"[information technology, innovation, literatur...",the purpose of this paper be to review and int...,"[information technology, innovation, literatur...","[information technology, literature review, in...","[innovation, information technology, literatur...","[group level, innovation, role, individual, to...","[publish, integrate, integrative, integration,...","[research publish focus, support innovation pu...","{NOUN, NOUN+NOUN}","[group level, innovation, role, topic]","[group level, innovation, role, individual, to...","[innovation publish, support innovation, group...","[support innovation publish, information syste...","[publish, integrate, integration, research, st...","[publish, integrate, integrative, integration,..."
2,many start-ups are in search of cooperation pa...,"[start-up cooperation, start-up performance]",many start-up be in search of cooperation part...,"[start-up cooperation, start-up performance]","[cooperation, start-up cooperation, cooperatio...","[cooperation, start-up performance, start-up c...","[cooperation behavior, start-, use, build, per...","[study, cooperation, base, start, factor, scal...","[past study identify, lack empirical research,...","{NOUN, NOUN+NOUN, ADJ+NOUN}","[cooperation behavior, start-, use, build, per...","[cooperation behavior, start-, use, build, per...",[],"[past study identify, past study focus, measur...","[study, cooperation, base, start, factor, scal...","[study, cooperation, base, start, factor, scal..."
3,"digital transformations are changing society, ...","[business processes, innovation]",digital transformation be change society and t...,"[business process, innovation]","[digital change, innovation, cooperative model]","[business process, innovation, digital change,...","[certain level, case company, change, digitali...","[change society, digital transformation, accor...","[process maturity level, process maturity mode...","{NOUN, NOUN+NOUN, ADJ+NOUN}","[certain level, case company, change, digitali...","[certain level, case company, change, digitali...","[process maturity, innovation level, business ...","[process maturity level, process maturity mode...","[change society, digital transformation, accor...","[change society, digital transformation, accor..."
4,the article identifies how collaborations with...,"[digital innovation, industry 4.0, innovation ...",the article identify how collaboration with st...,"[digital innovation, industry 4.0, innovation ...","[digital innovation, r d collaboration, open i...","[innovation practice, innovation ecosystem, di...","[startup, collaboration, industry 4, brazil]","[article identify, low maturity stage, theoret...","[international innovation center, open innovat...","{NOUN, NOUN+NOUN, ADJ+NOUN, NOUN+NOUN+NOUN, NO...","[startup, collaboration, industry 4, brazil]","[startup, collaboration, industry 4, brazil]","[government development agency, company univer...","[international innovation center, open innovat...",[use],"[article ident

все, мы собрали полный фрейм данных. Оставляем этот несчастный датафрейм в покое и идем оценивать, а насколько мы вообще хорошо сработали.

## 5

как тяжело смотреть на губы, которые не можешь поцеловать, и метрики, которые не можешь импортировать из sklearn. Пойдем тогда обратимся к формальному определению полноты и точности, чтобы понять, какие именно вещи мы хотим оценивать в задаче выбора ключевых слов.

In [1407]:
#how many relevant items are selected?
#selected relevant items/all relevant items
def recall_metric(dataframe,model):
    recall_kws = []
    for num in range(len(dataframe)):
        recall_kws.append(len([x for x in dataframe[model][num] if x in dataframe['gold_standard'][num]])/len(dataframe['gold_standard'][num]))
    return recall_kws

In [1408]:
#how many selected items are relevant?
#relevant selected items/all selected items
def precision_metric(dataframe,model):
    prec_kws = []
    for num in range(len(dataframe)):
        if len(dataframe[model][num]) == 0:
            prec_kws.append(float(0))
        else:
            prec_kws.append(len([x for x in dataframe[model][num] if x in dataframe['gold_standard'][num]])/len(dataframe[model][num]))
    return prec_kws

да-да, подстраховались, чтобы не делить на ноль. Полноте это не надо - там в делителе длина золотого стандарта, а он нигде не пустой.

новый датафрейм! О-бо-жаю датафреймы. Засунем сюда оценки полноты и точности всех трех наших моделек - без учета частей речи, с учетом локальных частеречных молдов и с учетом полных частеречных молдов.

In [1409]:
metrics_frame = pd.DataFrame({'RAKE_recall': recall_metric(data, 'RAKE'),
                            'RAKE_precision': precision_metric(data, 'RAKE'),
                            'RAKE_molded_loc_precision': precision_metric(data, 'RAKE_molded_loc'),
                            'RAKE_molded_full_precision': precision_metric(data, 'RAKE_molded_full'),
                            'YAKE_recall': recall_metric(data, 'YAKE'),
                            'YAKE_precision': precision_metric(data, 'YAKE'),
                            'YAKE_molded_loc_precision': precision_metric(data, 'YAKE_molded_loc'),
                            'YAKE_molded_full_precision': precision_metric(data, 'YAKE_molded_full'),
                            'textrank_recall': recall_metric(data, 'textrank'),
                            'textrank_precision': precision_metric(data, 'textrank'),
                            'textrank_molded_loc_precision': precision_metric(data, 'textrank_molded_loc'),
                            'textrank_molded_full_precision': precision_metric(data, 'textrank_molded_full')})

не позволяя себе ни минуты на отдых, идем вычислять для каждого случая ф-число - снова подстраховавшись, чтобы не поделить на ноль, - и вставляем в тот же датафрейм на удачные места.

In [1410]:
#(2*recall*precision)/recall+precision
def f_score_metric(dataframe, model):
    f_score_list = []
    f_score_list_loc = []
    f_score_list_full = []
    for num in range(len(dataframe)):
        sum_metrics = float(dataframe[f'{model}_recall'][num])+float(dataframe[f'{model}_precision'][num])
        sum_metrics_loc = float(dataframe[f'{model}_recall'][num])+float(dataframe[f'{model}_molded_loc_precision'][num])
        sum_metrics_full = float(dataframe[f'{model}_recall'][num])+float(dataframe[f'{model}_molded_full_precision'][num])
        if sum_metrics == 0:
            f_score_list.append(float(0))
        else:
            f_score_list.append((2*float(dataframe[f'{model}_recall'][num])*float(dataframe[f'{model}_precision'][num]))/(sum_metrics))
        if sum_metrics_loc == 0:
            f_score_list_loc.append(float(0))
        else:
            f_score_list_loc.append((2*float(dataframe[f'{model}_recall'][num])*float(dataframe[f'{model}_molded_loc_precision'][num]))/(sum_metrics_loc))
        if sum_metrics_full == 0:
            f_score_list_full.append(float(0))
        else:
            f_score_list_full.append((2*float(dataframe[f'{model}_recall'][num])*float(dataframe[f'{model}_molded_full_precision'][num]))/(sum_metrics_full))
    return f_score_list, f_score_list_loc, f_score_list_full

In [1411]:
metrics_frame.insert(4,'RAKE_f_score',f_score_metric(metrics_frame, 'RAKE')[0])
metrics_frame.insert(5, 'RAKE_m_f_score', f_score_metric(metrics_frame, 'RAKE')[1])
metrics_frame.insert(6, 'RAKE_mf_f_score', f_score_metric(metrics_frame, 'RAKE')[2])
metrics_frame.insert(11, 'YAKE_f_score', f_score_metric(metrics_frame, 'YAKE')[0])
metrics_frame.insert(12, 'YAKE_m_f_score', f_score_metric(metrics_frame, 'YAKE')[1])
metrics_frame.insert(13, 'YAKE_mf_f_score', f_score_metric(metrics_frame, 'YAKE')[2])
metrics_frame.insert(18, 'textrank_f_score', f_score_metric(metrics_frame, 'textrank')[0])
metrics_frame.insert(19, 'textrank_m_f_score', f_score_metric(metrics_frame, 'textrank')[1])
metrics_frame.insert(20, 'textrank_mf_f_score', f_score_metric(metrics_frame, 'textrank')[2])

In [1412]:
metrics_frame

,RAKE_recall,RAKE_precision,RAKE_molded_loc_precision,RAKE_molded_full_precision,RAKE_f_score,RAKE_m_f_score,RAKE_mf_f_score,YAKE_recall,YAKE_precision,YAKE_molded_loc_precision,YAKE_molded_full_precision,YAKE_f_score,YAKE_m_f_score,YAKE_mf_f_score,textrank_recall,textrank_precision,textrank_molded_loc_precision,textrank_molded_full_precision,textrank_f_score,textrank_m_f_score,textrank_mf_f_score
0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.333,0.15,0.429,0.176,0.207,0.375,0.231,0.111,0.167,0.167,0.167,0.133,0.133,0.133
1,0.333,0.200,0.250,0.200,0.250,0.286,0.250,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.167,0.200,0.200,0.200,0.182,0.182,0.182,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.167,0.083,0.111,0.091,0.111,0.133,0.118
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.250,0.05,0.250,0.091,0.083,0.250,0.133,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.143,0.250,0.250,0.250,0.182,0.182,0.182,0.143,0.05,0.143,0.062,0.074,0.143,0.087,0.000,0.000,0.000,0.000,0.000,0.000,0.000
5,0.333,0.429,0.429,0.429,0.375,0.375,0.375,0.111,0.05,1.000,0.083,0.069,0.200,0.095,0.111,0.091,0.100,0.091,0.100,0.105,0.100
6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.167,0.05,0.143,0.071,0.077,0.154,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000
7,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.333,0.15,0.500,0.250,0.207,0.400,0.286,0.000,0.000,0.000,0.000,0.000,0.000,0.000
8,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.571,0.20,0.500,0.400,0.296,0.533,0.471,0.143,0.167,1.000,0.200,0.154,0.250,0.167
9,0.222,1.000,1.000,1.000,0.364,0.364,0.364,0.111,0.05,0.125,0.100,0.069,0.118,0.105,0.000,0.000,0.000,0.000,0.000,0.000,0.000


как...красиво. Но не очень информативно. Найдем средний результат каждой модели на каждом сете данных и оформим как процент.

In [1413]:
metric_stats = pd.DataFrame({'metric': metrics_frame.columns.to_list(),
                             'score(%)': [round(x*100,2) for x in metrics_frame.mean().to_list()]})

In [1414]:
metric_stats

,metric,score(%)
0,RAKE_recall,12.48
1,RAKE_precision,23.38
2,RAKE_molded_loc_precision,25.99
3,RAKE_molded_full_precision,23.97
4,RAKE_f_score,14.19
5,RAKE_m_f_score,15.10
6,RAKE_mf_f_score,14.47
7,YAKE_recall,23.41
8,YAKE_precision,7.89
9,YAKE_molded_loc_precision,33.42


что же мы видим?

- молды по полному сету частеречных сочетаний незначительно повышают точность и ф-число и не меняют полноту
- молд по локальному сету частеречных сочетаний значительно повышает точность -> и повышает ф-число
- максимальная достигнутая точность: УАКЕ на локальном сете (33.42%); минимальная - textrank без молда (7.18%)
- максимальная достигнутая полнота: УАКЕ (23.41%); минимальная - textrank (11.01%)
- максимальное ф-число: УАКЕ на локальном сете (23.83%); минимальное - textrank без молда (8.15%)

чтобы нам было удобнее на это все смотреть, сделаем себе подобие матрицы ошибок для каждой из моделей, которые чем-то отличились - в хорошую или в плохую сторону.

In [1415]:
research_frame = data[['gold_standard' , 'YAKE_molded_loc' , 'textrank_molded_loc', 'RAKE_molded_loc']]

In [1416]:
TP_textrank = []
FP_textrank = []
FN_textrank = []

for num in range(len(research_frame)):
    TP_textrank.append(set(research_frame['gold_standard'][num])&set(research_frame['textrank_molded_loc'][num]))
    FP_textrank.append(set(research_frame['textrank_molded_loc'][num])-set(research_frame['gold_standard'][num]))
    FN_textrank.append(set(research_frame['gold_standard'][num])-set(research_frame['textrank_molded_loc'][num]))

confusion_frame_textrank = pd.DataFrame({'правда': TP_textrank,
                                'лишнее': FP_textrank,
                                'не выделено': FN_textrank})

confusion_frame_textrank

,правда,лишнее,не выделено
0,{innovation},"{standpoint, platform use, inter, compress ant...","{outcome, innovation input, digital, innovatio..."
1,{},"{publish, integrate, author focus, study, rese...","{information technology, innovation, literatur..."
2,{cooperation},"{base, incumbent, questionnaire, use, factor, ...","{incumbent firm, start-up cooperation, coopera..."
3,{},"{framework, accord, digital transformation, pr...","{business process, innovation, digital change,..."
4,{},{use},"{innovation practice, innovation ecosystem, di..."
5,{digiware},"{include, recognise, business, technology deve...","{farm management, digital innovation, advisory..."
6,{},"{contribute, theory, rapid development, status...","{ground theory, in-depth interview, internet, ..."
7,{},"{academia, routine, clinical practicality, pra...","{pet, ddmc, real-time, motion correction, real..."
8,{communication technology},{},"{e-school project, computer literacy, informat..."
9,{},{},"{digital intervention, low resource set, menta..."


что ж, обладатель минимальных значений по всем трем метрикам - textrank без молда. Посмотрим, что у него не так.
- слишком сильно любит однословные теговые блоки
- в принципе неплох, но при любой возможности использует глаголы
- есть вероятность, что теггеру мешает маленький размер наших единиц. У английского языка непросто догадаться о части речи многих слов без контекста, а в ключевом блоке контекст это зачастую слово-два, если повезет. Вот он и не справляется с тем, чтобы все глаголы и даже прилагательные выкинуть. Но надо покрутить молд - потыкать, правильно ли он с этой моделью работает, потому что прилагательные явно не должны проходить фильтрацию, как и слово integrate там, где глаголов в молде вообще нет...
- похоже, не видит слов с дефисами

In [1417]:
TP_YAKE_molded_loc = []
FP_YAKE_molded_loc = []
FN_YAKE_molded_loc = []

for num in range(len(research_frame)):
    TP_YAKE_molded_loc.append(set(research_frame['gold_standard'][num])&set(research_frame['YAKE_molded_loc'][num]))
    FP_YAKE_molded_loc.append(set(research_frame['YAKE_molded_loc'][num])-set(research_frame['gold_standard'][num]))
    FN_YAKE_molded_loc.append(set(research_frame['gold_standard'][num])-set(research_frame['YAKE_molded_loc'][num]))

confusion_frame_YAKE_molded_loc = pd.DataFrame({'правда': TP_YAKE_molded_loc,
                                'лишнее': FP_YAKE_molded_loc,
                                'не выделено': FN_YAKE_molded_loc})

display(confusion_frame_YAKE_molded_loc)

,правда,лишнее,не выделено
0,"{business process, digital innovation, innovat...","{issue accord, issue paper, digital technology...","{innovation, outcome, innovation input, digita..."
1,{},"{author focus, support innovation, group level...","{information technology, innovation, literatur..."
2,{},{},"{cooperation, start-up performance, incumbent ..."
3,{business process},"{process maturity, innovation partner, innovat...","{innovation, digital change, cooperative model}"
4,{open innovation},"{incubator characterise, government developmen...","{innovation practice, innovation ecosystem, di..."
5,{farm adviser},{},"{farm management, digiware, digital innovation..."
6,{public welfare},"{internet platform, welfare project, welfare o...","{ground theory, in-depth interview, internet, ..."
7,"{hardware-driven motion correction, data-drive...","{medical physics integrate, real-time acquisit...","{pet, ddmc, real-time, real-time ddmc, data-dr..."
8,"{computer literacy, communication technology, ...","{mature school, theoretical review, school pro...","{informatic curriculum, digitally mature schoo..."
9,{mental health care},"{low resource setting, digital intervention su...","{digital intervention, low resource set, menta..."


наш чемпион - УАКЕ с локальным молдом! Действительно хорош.
- периодически даже не выделяет лишнего! действительно мало лишнего выделяет - правда, сочетает это с тем, что вообще ничего не выделяет.
- плачет каждый раз, когда длина тега меньше трех. Героически выделил ключевые блоки из двух слов, ключевые блоки из одного слова гордо игнорирует.
- спокойно относится к дефисам

давайте хоть глянем, чем там РАКЕ занимался...

In [1418]:
TP_RAKE_molded_loc = []
FP_RAKE_molded_loc = []
FN_RAKE_molded_loc = []

for num in range(len(research_frame)):
    TP_RAKE_molded_loc.append(set(research_frame['gold_standard'][num])&set(research_frame['RAKE_molded_loc'][num]))
    FP_RAKE_molded_loc.append(set(research_frame['RAKE_molded_loc'][num])-set(research_frame['gold_standard'][num]))
    FN_RAKE_molded_loc.append(set(research_frame['gold_standard'][num])-set(research_frame['RAKE_molded_loc'][num]))

confusion_frame_RAKE_molded_loc = pd.DataFrame({'правда': TP_RAKE_molded_loc,
                                'лишнее': FP_RAKE_molded_loc,
                                'не выделено': FN_RAKE_molded_loc})

display(confusion_frame_RAKE_molded_loc)

,правда,лишнее,не выделено
0,{},"{digital technology, purpose}","{innovation, outcome, innovation input, digita..."
1,{innovation},"{topic, group level, role}","{information technology, literature review}"
2,{cooperation behavior},"{start-, use, build, performance}","{incumbent firm, start-up cooperation, coopera..."
3,{},"{study, change, use, digitalise, model, relati...","{business process, innovation, digital change,..."
4,{startup},"{industry 4, brazil, collaboration}","{innovation practice, innovation ecosystem, di..."
5,"{farm adviser, digital innovation, digiware}","{recognise, engage, service, digital tool}","{farm management, advisory service, agricultur..."
6,{},"{internet platform, integration}","{ground theory, in-depth interview, internet, ..."
7,{},"{opportunity, development}","{pet, ddmc, real-time, motion correction, real..."
8,{},{},"{communication technology, e-school project, d..."
9,"{digital technology, mental health}",{},"{digital intervention, low resource set, menta..."


тоже обладающий неплохими показателями метрик РАКЕ с локальным молдом! посмотрим, что он делает.
- РАКЕ плохо справляется с дефисами - он их все-таки считает поводом разбить слово. Возможно, у него вшит собственный механизм разбивки-токенизации, который решает это все убрать; тем не менее, один раз с дефисом он справился (17) - не опознал слово или его составляющие и оставил в покое?
- выкидывать научился в основном прилагательные, с существительными и глаголами все еще путается - но это английский язык, что тут поделаешь. Это надо деление на части речи у спайси покрутить, или в наши молды потыкать, как и у УАКЕ
- действительно не любит выделять по три слова в ключевом блоке.
- сложные цифры (4.0) ему тоже не нравятся - это тоже указывает в сторону вшитого токенизатора со своими правилами

что исправить, чтобы все стало лучше?
- для РАКЕ: добиться того, чтобы он выделял ключевые блоки с большей длиной и не крутился вечно вокруг однословных - впрочем, возможно, это у нас маленькие кусочки текстов и мало токенов; убедить его не токенизировать по дефисам и вообще дополнительно не токенизировать
- для УАКЕ: ничего, он умничка, чмок в лобик
- для textrank: узнать, что же там происходит с частеречным делением, что мешает молду фильтровать у него вечные глаголы-прилагательные; тоже как-то пихнуть его в сторону более длинных ключевых блоков - возможно, присваивать более длинным блокам чуточку большие веса? Не факт, что это технически выполнимо, и если даже и да, нужно будет очень точно определить коэффициент домножения весов, чтобы он не начал компульсивно собирать все длинные цепочки, что можно придумать.

+ для всех: возможно, деление на части речи стоит производить еще когда текст не очищен и не лемматизирован. Куда-то туда можно воткнуть этот этап; впрочем, он особо не поможет, когда моделькам придется расставлять части речи своих кандидатов - у кандидатов-то контекста нет. Можно, например, для каждого кандидата искать его же в изначальном тексте содержания и вытаскивать часть речи оттуда - там у него еще есть полный контекст.